<a href="https://colab.research.google.com/github/agarwalshashwat/Machine_Learning/blob/main/ML_Major_Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('Information.csv', encoding = 'latin-1',error_bad_lines=False, engine="python")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17998 entries, 0 to 17997
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               17998 non-null  int64  
 1   _golden                17998 non-null  bool   
 2   _unit_state            17998 non-null  object 
 3   _trusted_judgments     17998 non-null  int64  
 4   _last_judgment_at      17998 non-null  object 
 5   gender                 17913 non-null  object 
 6   gender:confidence      17975 non-null  float64
 7   profile_yn             17998 non-null  object 
 8   profile_yn:confidence  17998 non-null  float64
 9   created                17998 non-null  object 
 10  description            14663 non-null  object 
 11  fav_number             17998 non-null  int64  
 12  gender_gold            0 non-null      float64
 13  link_color             17998 non-null  object 
 14  name                   17998 non-null  object 
 15  pr

In [8]:
df = df.drop(['_unit_id','_golden','_last_judgment_at','_unit_state','_trusted_judgments','profile_yn','profile_yn:confidence','created','fav_number','gender_gold','link_color','name','profile_yn_gold','profileimage','retweet_count','sidebar_color','tweet_coord','tweet_count','tweet_created','tweet_id','tweet_location','user_timezone'], axis = 1) 

In [9]:
df = df[df['gender:confidence'] == 1]

In [10]:
male = df[df['gender'] == 'male']
female = df[df['gender'] == 'female']
df = male.append(female)

In [11]:
df.head()

,gender,gender:confidence,description,text
0,male,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,1.0,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
3,male,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
7,male,1.0,The secret of getting ahead is getting started.,Gala Bingo clubs bought for å£241m: The UK's l...
17,male,1.0,"Over enthusiastic F1 fan. Model collector, mu...",@coolyazzy94 Ditto - I'm still learning the fa...


In [12]:
df['description'] = df['description'].astype(str)
df['text'] = df['text'].astype(str)

In [13]:
df['data'] = df[['description', 'text']].apply(lambda x: ' '.join(x), axis=1)

In [14]:
df = df.reset_index()

In [15]:
del df['index']

In [16]:
df.head()

,gender,gender:confidence,description,text,data
0,male,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm. Robbie E Responds To Cri...
1,male,1.0,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,I'm the author of novels filled with family dr...
2,male,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
3,male,1.0,The secret of getting ahead is getting started.,Gala Bingo clubs bought for å£241m: The UK's l...,The secret of getting ahead is getting started...
4,male,1.0,"Over enthusiastic F1 fan. Model collector, mu...",@coolyazzy94 Ditto - I'm still learning the fa...,"Over enthusiastic F1 fan. Model collector, mu..."


In [24]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [25]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [26]:
lemma = WordNetLemmatizer()

In [27]:
corpus = []
for i in range(0,len(df)):
    words = re.sub('[^a-zA-Z]',' ',df['data'][i])
    words = words.lower()
    words = words.split()
    words = [lemma.lemmatize(word) for word in words if word not in stopwords.words('english')]
    words = ' '.join(words)
    corpus.append(words)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvec= TfidfVectorizer(max_features=5000)

In [29]:
x = tfvec.fit_transform(corpus).toarray()

In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [31]:
df['gender_label'] = le.fit_transform(df['gender'])

In [32]:
y = df[['gender_label']].values

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8)

In [35]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((7120, 5000), (1780, 5000), (7120, 1), (1780, 1))

In [36]:
from sklearn.naive_bayes import MultinomialNB

In [37]:
gender_detect_nb = MultinomialNB().fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
y_pred_nb = gender_detect_nb.predict(x_test)

In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
acurracy_cm = confusion_matrix(y_test,y_pred_nb)

In [41]:
acurracy_cm

array([[762, 186],
       [368, 464]])

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
accuracy_score(y_test,y_pred_nb)

0.6887640449438203